In [ ]:
"""
Created on Mon Oct 21 16:13 2024

Prepare what's needed for eORCA1 on the ice-shelf side

@author: Clara Burgard
"""

In [80]:
import xarray as xr
import numpy as np
from cdo import Cdo
from tqdm.notebook import tqdm


In [18]:
%matplotlib qt5

In [19]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 2.0.3


In [20]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

REMAP THE ICE SHELF CONCENTRATION

In [27]:
dom_cfg_path = inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'
mask_file_path = inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc'
grid_file_path = inputpath_interim + 'grid_eORCA1_T.nc'

In [22]:
cdo.selvar('isf_conc', input = mask_file_path, output = inputpath_interim + 'isf_conc_only_2km.nc')

'/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/isf_conc_only_2km.nc'

In [29]:
cdo.remapbil(grid_file_path, input = inputpath_interim + 'isf_conc_only_2km.nc', output = inputpath_interim + 'isf_conc_only_targetgrid.nc')

'/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/isf_conc_only_targetgrid.nc'

In [49]:
isf_conc_2km = xr.open_dataset(inputpath_interim + 'isf_conc_only_2km.nc')['isf_conc']

REMAP THE IMBIE MASK

In [30]:
cdo.selvar('Iceshelf_extrap', input = mask_file_path, output = inputpath_interim + 'isfmask_only_2km.nc')

'/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/isfmask_only_2km.nc'

In [31]:
cdo.remapnn(grid_file_path, input = inputpath_interim + 'isfmask_only_2km.nc', output = inputpath_interim + 'isfmask_only_targetgrid.nc')

'/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/isfmask_only_targetgrid.nc'

CREATE MASK_VARIABLE

In [40]:
domain_cfg = xr.open_dataset(dom_cfg_path)
isfmask = xr.open_dataset(inputpath_interim + 'isfmask_only_targetgrid.nc')
isfconc = xr.open_dataset(inputpath_interim + 'isf_conc_only_targetgrid.nc')

In [97]:
ds_isfNEMO = xr.Dataset()

ds_isfNEMO['mask_isf'] = isfmask['Iceshelf_extrap'].where(domain_cfg['isf_draft'] > 0)
ds_isfNEMO['mask_isf_onlyisfconc1'] = ds_isfNEMO['mask_isf'].where(isfconc['isf_conc'] > 0.9999)


CREATE MASK_ISF_OPEN

In [89]:
file_Justine = xr.open_dataset(mask_file_path).drop('quantile')

In [58]:
for iid in file_Justine.ID:
    print(iid.values,file_Justine['NAME'].sel(ID=iid).values)

1 Jelbart
2 Fimbul
3 
4 Vigrid
5 Nivl
6 
7 Lazarev
8 Borchgrevink
9 Baudouin
10 
11 Prince_Harald
12 
13 Shirase
14 Hannan/Telen/Skallen
15 Rayner_Thyer
16 Zubchatyy/Porter/Myers
17 WilmaRobertDowner
18 Edward_VIII
19 
20 Utsikkar/Mulebreen/Cirque_Fjord/Hoseason/Rund_Bay
21 Amery
22 Publications
23 Sorsdal/Hovde/Flatnes/Dalk
24 West
25 Helen
26 Shackleton
27 Tracy_Tremenchus
28 
29 Conger_Glenzer
30 
31 Vincennes_Bay/Underwood
32 Williamson/Whittle/FoxE
33 Totten
34 Moscow_University
35 
36 Voyeykov
37 
38 Holmes
39 Frost
40 May_Glacier/Morse/Sandford
41 Dibble
42 WattBay/Zelee/Astrolabe/Liotard/Francais/Marret/Commandant_Charcot//PourquoiPas
43 Mertz
44 Ainsworth/Fisher
45 Ninnis
46 Deakin/HornBluff
47 Cook
48 Andreyev/Slava
49 Lauritzen
50 Drury
51 Matusevitch
52 Suvorov/Gillet/Noll/Paternostro/McLeod
53 Rennick
54 Chugunov/Barber/Gannutz
55 Lillie
56 Arneb/Manhaul/Ironside/Quatermain_Point/Moubray/Dennistoun/SmithInlet//Kirkby
57 Tucker
58 Mandible_Cirque
59 Mariner
60 Parker/Falkne

In [90]:
ID_open_list = [1,2,21,66,67,117,124,125,127,128]

In [91]:
file_Justine_open = file_Justine.sel(ID=ID_open_list)
file_Justine_closed = file_Justine.drop_sel(ID=ID_open_list)

In [98]:
domain_cfg_closed =  domain_cfg.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    for vv in ['bottom_level','top_level','isf_draft','bathy_metry']:
        domain_cfg_closed[vv] = domain_cfg_closed[vv].where(ds_isfNEMO['mask_isf'].drop_vars(['glamt', 'gphit']) != id_closed.values, 0)

  0%|          | 0/123 [00:00<?, ?it/s]

In [ ]:
# create mask_isf_open
ds_isfNEMO['mask_isf_open'] = 

In [ ]:
# make maps of zmin and zmax on the front (for ice shelves we want to close)
# identify front
# create mask_isf_closed

In [ ]:
# distribute melt at the front (normalised by cell size)

In [ ]:
# create mask_isf_for_application

In [ ]:
# open ice shelf mask
ds_isf=xr.open_dataset('Mask_Iceshelf_IMBIE2_v2.nc')